## SRJ 

In [1]:
import pennylane as qml
from pennylane import qchem
from pennylane import numpy as np
from itertools import chain
import time
import re
from scipy.optimize import minimize
ash_excitation = []
energies = []
excitations= []

X = qml.PauliX
Y = qml.PauliY
Z = qml.PauliZ
I = qml.Identity






def ags_exact(symbols, coordinates,electrons, orbitals, active_electrons, active_orbitals):
    print('Using active space, check if you change the H accordingly')
    #H, qubits = qml.qchem.molecular_hamiltonian(symbols, coordinates, basis="sto-3g", method="pyscf",active_electrons=active_electrons, active_orbitals=active_orbitals)
    core, active = qml.qchem.active_space(electrons, orbitals, mult=5, active_electrons=active_electrons, active_orbitals=active_orbitals) # Spatial orbitals
    print('core orbitals:', core)
    print('active orbitals:', active)
    #print(H)
    return core, active

In [2]:
symbols  = [ 'Fe', 'S']
print('H2O-0.6A-GS+ES-DBS')
r_bohr = 1.8897259886 
#r_bohr = 1.0
coordinates = np.array([[0.0,0.0, 0.0], [0.0, 0.0, 1.826*r_bohr]])

print(coordinates)

electrons = 42  # 26 from Fe and 16 from S
orbitals = 54 #Spin orbitals   27 spatial from 
charge = 0

active_electrons = 6 #No of electrons
active_orbitals = 6 #Spatial orbitals

core, active = ags_exact(symbols, coordinates, electrons, orbitals, active_electrons, active_orbitals)

H2O-0.6A-GS+ES-DBS
[[0.         0.         0.        ]
 [0.         0.         3.45063966]]
Using active space, check if you change the H accordingly
core orbitals: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17]
active orbitals: [18, 19, 20, 21, 22, 23]


In [6]:
len(core) + len(active)

24

## Got this from pennylane

In [22]:
electrons = 42 # 26 From Fe + 16 from S
orbitals = 54 #Spin orbitals - 27 * 2
active_electrons = 6
active_orbitals = 6
ncore_orbs = (electrons - active_electrons) // 2
print('ncore_obs', ncore_orbs)
mult = 5

ncore_obs 18


In [17]:
ncore_orbs = (electrons - active_electrons) // 2
core = list(range(ncore_orbs))
print(core)

if active_orbitals is None:
    active = list(range(ncore_orbs, orbitals))
else:
    if active_orbitals <= 0:
        raise ValueError(
            f"The number of active orbitals ({active_orbitals}) " f"has to be greater than 0."
        )

    if ncore_orbs + active_orbitals > orbitals:
        raise ValueError(
            f"The number of core ({ncore_orbs}) + active orbitals ({active_orbitals}) cannot "
            f"be greater than the total number of orbitals ({orbitals})"
        )

    homo = (electrons + mult - 1) / 2
    if ncore_orbs + active_orbitals <= homo:
        raise ValueError(
            f"For n_active_orbitals={active_orbitals}, there are no virtual orbitals "
            f"in the active space."
        )

    active = list(range(ncore_orbs, ncore_orbs + active_orbitals))
    print(active)




[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17]
[18, 19, 20, 21, 22, 23]


## Original code

In [3]:
import basis_set_exchange as bse
print('Check if I am using ANO-RCC-MB basis set')
print('I am commenting the newton method ')
basis_dict = bse.get_basis('ANO-RCC-MB', elements=['Fe', 'S'], fmt='nwchem')

import numpy as np
from pyscf import gto, scf, mcscf
from pyscf import gto, ao2mo, scf
import pennylane as qml

#print('Using newton method')
# Your original setup (unchanged) 
mol_pyscf = gto.M(
    atom = 'Fe 0 0 0; S 0 0 1.826',
    basis = basis_dict,
    spin = 4)
rhf = scf.ROHF(mol_pyscf)
rhf.max_cycle = 2000
rhf.init_guess = "minao"
rhf.level_shift = 1.0
#rhf.damp = 0.2
#rhf.diis_start_cycle = 5

#rhf = rhf.newton() 
rhf.kernel()

import pennylane as qml 
from pennylane import qchem


one_ao = mol_pyscf.intor_symmetric("int1e_kin") + mol_pyscf.intor_symmetric("int1e_nuc")
two_ao = mol_pyscf.intor("int2e_sph")


one_mo = np.einsum("pi,pq,qj->ij", rhf.mo_coeff, one_ao, rhf.mo_coeff, optimize=True)
two_mo = ao2mo.incore.full(two_ao, rhf.mo_coeff)

#print('Shape of one mo', one_mo.shape)
#print('Shape of two mo', two_mo.shape)
core_constant = np.array([rhf.energy_nuc()])
#print('The value of core:', core_constant)

# convert the two-electron integral tensor to the physicists’ notation
two_mo = np.swapaxes(two_mo, 1, 3)
#print('Shape after swapping axes', two_mo.shape)

#Core and active are received from the previous code
core = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17]
active = [18, 19, 20, 21, 22, 23]

total_orb = len(core) + len(active)
print('Total orbitals are',total_orb)



# define the active space and recompute the integrals
def pyscf_integral(core, active, one_mo, two_mo, core_constant):
    if core and active:
        for i in core:
            #print('The value of i going in ', i)
            core_constant = core_constant + 2 * one_mo[i][i]
            #print('core cosntant', core_constant)
            for j in core:
                #print('Value of i and j ', i,j)
                core_constant = core_constant + 2 * two_mo[i][j][j][i] - two_mo[i][j][i][j]

        for p in active:
            for q in active:
                for i in core:
                    #print(f'The value of p and q', {p}, {q})
                    one_mo[p, q] = one_mo[p, q] + (2 * two_mo[i][p][q][i] - two_mo[i][p][i][q])

        one_mo = one_mo[qml.math.ix_(active, active)]
        two_mo = two_mo[qml.math.ix_(active, active, active, active)]
        print('Within pyscf integral one_mo shape', one_mo.shape)
        print('Within pyscf integral one_mo shape', one_mo.shape)
    return core_constant, one_mo, two_mo

core_constant, one_mo, two_mo = pyscf_integral(core, active, one_mo,two_mo, core_constant)


H_fermionic = qml.qchem.fermionic_observable(core_constant, one_mo, two_mo, cutoff=1e-20)

H = qml.jordan_wigner(H_fermionic)

print(H)

Check if I am using ANO-RCC-MB basis set
I am commenting the newton method 
converged SCF energy = -1655.20119274802
Total orbitals are 24
Within pyscf integral one_mo shape (6, 6)
Within pyscf integral one_mo shape (6, 6)
(-1654.0228380641202+0j) * I(0) + (0.18059721737727963+0j) * Z(0) + (-0.00450117800047325+0j) * (Y(0) @ Z(1) @ Y(2)) + (-0.00450117800047325+0j) * (X(0) @ Z(1) @ X(2)) + -5.221517662690189e-16j * (Y(0) @ Z(1) @ Z(2) @ Z(3) @ X(4)) + (0.03837975672927615+0j) * (Y(0) @ Z(1) @ Z(2) @ Z(3) @ Y(4)) + (0.03837975672927615+0j) * (X(0) @ Z(1) @ Z(2) @ Z(3) @ X(4)) + 5.221517662690189e-16j * (X(0) @ Z(1) @ Z(2) @ Z(3) @ Y(4)) + (-0.09246791318067471+0j) * Z(2) + (0.07678491313428998+0j) * (Z(0) @ Z(2)) + -1.809533425878307e-16j * (Y(2) @ Z(3) @ X(4)) + (-0.0007888530774776072+0j) * (Y(2) @ Z(3) @ Y(4)) + (-0.0007888530774776072+0j) * (X(2) @ Z(3) @ X(4)) + 1.809533425878307e-16j * (X(2) @ Z(3) @ Y(4)) + (-0.001083874881190823+0j) * (Z(0) @ Y(2) @ Z(3) @ Y(4)) + (-0.0010838748

1. In the previous step, the converged SCF energy is different

1. Argument: 
    Full space Hamiltonian based HF energy (54 qubits) =! Reduced space HF energy (12 qubits). It doesn't need to match 

1. atom, minao, hcore. In that atom and hcore doesn't work

2. Added cut off to fermionic observable, no change. 

3. Maxcycle = 2000, no change

4. Changed basis set - def2-SVP, yet the values stays different. 

1.  Energy produced by H should match the ROHF. 

# Using pennylane format - HF state

In [ ]:




#Check the possible excitations
import pennylane as qml
from pennylane import qchem
from pennylane import numpy as np
from itertools import chain
import time
import re
import warnings
#warnings.filterwarnings("ignore", category=np.ComplexWarning)
from scipy.optimize import minimize
optimizer = qml.AdamOptimizer(stepsize=0.5)
ash_excitation = []
energies = []
excitations= []
old_grad = []

X = qml.PauliX
Y = qml.PauliY
Z = qml.PauliZ
I = qml.Identity






def ags_exact(active_electrons, H, adapt_it):
    print('Using active space, check if you change the H accordingly')
    qubits = 12
    active_electrons = active_electrons
    #print(H)
    hf_state = qchem.hf_state(active_electrons, qubits)
    #hf_state = np.array([1,1,1,0,0,0,1,1,1,0,0,0])
    #Calculation of HF state
    dev = qml.device("lightning.qubit", wires=qubits)
    @qml.qnode(dev)
    def circuit(hf_state, active_electrons, qubits, H): 
        qml.BasisState(hf_state, wires=range(qubits))
        return qml.expval(H)   #Calculating the expectation value of the Hamiltonian
    
    # Commutator calculation for HF state
    @qml.qnode(dev)
    def commutator_0(H,w, k):  #H is the Hamiltonian, w is the operator, k is the basis state - HF state
        qml.BasisState(k, wires=range(qubits))
        res = qml.commutator(H, w)   #Calculating the commutator
        return qml.expval(res)
    
    # Commutator calculation for other states except HF state
    @qml.qnode(dev)
    def commutator_1(H,w, k): #H is the Hamiltonian, w is the operator, k is the basis state
        qml.StatePrep(k, wires=range(qubits))
        res = qml.commutator(H, w) #Calculating the commutator
        return qml.expval(res)

    #Energy calculation 
    @qml.qnode(dev)
    def ash(params, ash_excitation, hf_state, H):
        [qml.PauliX(i) for i in np.nonzero(hf_state)[0]]  #Appln of HF state
        for i, excitation in enumerate(ash_excitation):
            if len(ash_excitation[i]) == 4:
                qml.FermionicDoubleExcitation(weight=params[i], wires1=list(range(ash_excitation[i][0], ash_excitation[i][1] + 1)), wires2=list(range(ash_excitation[i][2], ash_excitation[i][3] + 1)))
            elif len(ash_excitation[i]) == 2:
                qml.FermionicSingleExcitation(weight=params[i], wires=list(range(ash_excitation[i][0], ash_excitation[i][1] + 1)))
        return qml.expval(H)  #Calculating the expectation value of the Hamiltonian
    
    # Calculation of New state, same as the above function but with the state return
    dev1 = qml.device("lightning.qubit", wires=qubits)
    @qml.qnode(dev1)
    def new_state(hf_state, ash_excitation, params):
        [qml.PauliX(i) for i in np.nonzero(hf_state)[0]] #Applying the HF state
        for i, excitations in enumerate(ash_excitation):
            if len(ash_excitation[i]) == 4:
                qml.FermionicDoubleExcitation(weight=params[i], wires1=list(range(ash_excitation[i][0], ash_excitation[i][1] + 1)), wires2=list(range(ash_excitation[i][2], ash_excitation[i][3] + 1)))
            elif len(ash_excitation[i]) == 2:
                qml.FermionicSingleExcitation(weight=params[i], wires=list(range(ash_excitation[i][0], ash_excitation[i][1] + 1)))
        return qml.state()
    

    
    
    def cost(params):
        energy = ash(params, ash_excitation, hf_state, H)
        return energy

    #def callback(params):
        #print(f"Current parameters: {params}")
        #print(f"Current cost: {cost(params)}\n")
    

    print('HF state is', circuit(hf_state, active_electrons, qubits, H))
    singles, doubles = qml.qchem.excitations(active_electrons, qubits)

    op1 =  [qml.fermi.FermiWord({(0, x[0]): "+", (1, x[1]): "-"}) for x in singles]
    op2 =  [qml.fermi.FermiWord({(0, x[0]): "+", (1, x[1]): "+", (2, x[2]): "-", (3, x[3]): "-"})for x in doubles]
    operator_pool = (op1) + (op2)  #Operator pool - Singles and Doubles
    print('Total excitations are', len(operator_pool))
    states = [hf_state]
    params = np.zeros(len(ash_excitation), requires_grad=True) 


    

    for j in range(adapt_it):
        print('The adapt iteration now is', j)  #Adapt iteration
        max_value = float('-inf')
        max_operator = None
        k = states[-1] if states else hf_state  # if states is empty, fall back to hf_state
       
        for i in operator_pool:
            #print('The current excitation operator is', i)   #Current excitation operator - fermionic one
            w = qml.fermi.jordan_wigner(i)  #JW transformation
            if np.array_equal(k, hf_state): # If the current state is the HF state
                current_value = abs(2*(commutator_0(H, w, k)))      #Commutator calculation is activated  
            else:
                current_value = abs(2*(commutator_1(H, w, k)))      #For other states, commutator calculation is activated

            if current_value > max_value:
                max_value = current_value
                max_operator = i

        print(f"The highest operator value is {max_value} for operator {max_operator}")  #Highest operator value

        old_grad.append(max_value)  #Appending the old gradient value
        indices_str = re.findall(r'\d+', str(max_operator))
        excitations = [int(index) for index in indices_str]
        print('Highest gradient excitation is', excitations)
        ash_excitation.append(excitations) #Appending the excitations to the ash_excitation

        params = np.append(params, 0.0)  #Parameters initialization

        #Energy calculation
        result = minimize(cost, params, method='BFGS', tol = 1e-16, options = {'disp': False, 'maxiter': 1e8, 'gtol': 1e-12})

        print("Final updated parameters:", result.x)
        print("Final cost:", result.fun)

        params= (result.x)
        energies.append(result.fun)



        ostate = new_state(hf_state, ash_excitation, params)
        print(qml.draw(new_state, max_length=100)(hf_state,ash_excitation,params))
        gs_state = ostate
        states.append(ostate)
        
    return gs_state, params, ash_excitation, qubits, H, energies, old_grad






#electrons = 10  # 7 from N and 3 from H
#orbitals = 16
charge = 0

active_electrons = 6


gs_state, params, ash_excitation, qubits, H,energies, old_grad = ags_exact( active_electrons, H, adapt_it=50) #1 is used for params



print('The params after GS is',params)
print('Ash excitation after gs state:', ash_excitation)
print('Energies:', energies)
print('Old gradient:', old_grad)    


Using active space, check if you change the H accordingly
HF state is -1654.5304866257657
Total excitations are 117
The adapt iteration now is 0


/Users/s.poyyapakkam/miniconda3/envs/na/lib/python3.11/site-packages/pennylane_lightning/lightning_base/_serialize.py:359: ComplexWarning: Casting complex values to real discards the imaginary part
  coeffs = np.array(coeffs).astype(self.rtype)


The highest operator value is 0.06318841246221864 for operator a⁺(4) a⁺(5) a(10) a(11)
Highest gradient excitation is [4, 5, 10, 11]


/Users/s.poyyapakkam/miniconda3/envs/na/lib/python3.11/site-packages/pennylane_lightning/lightning_base/_serialize.py:359: ComplexWarning: Casting complex values to real discards the imaginary part
  coeffs = np.array(coeffs).astype(self.rtype)
/Users/s.poyyapakkam/miniconda3/envs/na/lib/python3.11/site-packages/pennylane_lightning/lightning_base/_serialize.py:359: ComplexWarning: Casting complex values to real discards the imaginary part
  coeffs = np.array(coeffs).astype(self.rtype)
/Users/s.poyyapakkam/miniconda3/envs/na/lib/python3.11/site-packages/pennylane_lightning/lightning_base/_serialize.py:359: ComplexWarning: Casting complex values to real discards the imaginary part
  coeffs = np.array(coeffs).astype(self.rtype)


Final updated parameters: [0.53223676]
Final cost: -1654.5390832780615
 0: ──X──────────────────────────────────┤  State
 1: ──X──────────────────────────────────┤  State
 2: ──X──────────────────────────────────┤  State
 3: ──X──────────────────────────────────┤  State
 4: ──X─╭FermionicDoubleExcitation(0.53)─┤  State
 5: ──X─├FermionicDoubleExcitation(0.53)─┤  State
10: ────├FermionicDoubleExcitation(0.53)─┤  State
11: ────╰FermionicDoubleExcitation(0.53)─┤  State
The adapt iteration now is 1


...

Final updated parameters: [ 2.8561103   0.05577781 -0.02214952  0.01458764  2.85135322 -0.06181664
  0.02326068  0.01782178  3.17219467 -3.13998837 -0.02709232 -0.01323763
 -3.38561077 -3.07823971]
Final cost: -1655.173490321495
 0: ──X──────────────────────────────────╭FermionicDoubleExcitation(0.06) ···
 1: ──X──────────────────────────────────├FermionicDoubleExcitation(0.06) ···
 2: ──X──────────────────────────────────├FermionicDoubleExcitation(0.06) ···
 3: ──X──────────────────────────────────├FermionicDoubleExcitation(0.06) ···
 4: ──X─╭FermionicDoubleExcitation(2.86)─├FermionicDoubleExcitation(0.06) ···
 5: ──X─├FermionicDoubleExcitation(2.86)─├FermionicDoubleExcitation(0.06) ···
 6: ────│────────────────────────────────│─────────────────────────────── ···
 7: ────│────────────────────────────────│─────────────────────────────── ···
 8: ────│────────────────────────────────│─────────────────────────────── ···
 9: ────│────────────────────────────────│───────────────────────────

/Users/s.poyyapakkam/miniconda3/envs/na/lib/python3.11/site-packages/pennylane_lightning/lightning_base/_serialize.py:359: ComplexWarning: Casting complex values to real discards the imaginary part
  coeffs = np.array(coeffs).astype(self.rtype)


The highest operator value is 0.006575238477769707 for operator a⁺(0) a⁺(1) a(8) a(9)
Highest gradient excitation is [0, 1, 8, 9]
Final updated parameters: [ 2.85438965  0.02886226 -0.013994    0.01572138  2.85246586 -0.05226368
  0.01999934  0.01982178  3.14701114 -3.14005242 -0.01073384 -0.0113344
 -3.38973988 -3.0816444   0.00844228]
Final cost: -1655.1735195557326
 0: ──X──────────────────────────────────╭FermionicDoubleExcitation(0.03) ···
 1: ──X──────────────────────────────────├FermionicDoubleExcitation(0.03) ···
 2: ──X──────────────────────────────────├FermionicDoubleExcitation(0.03) ···
 3: ──X──────────────────────────────────├FermionicDoubleExcitation(0.03) ···
 4: ──X─╭FermionicDoubleExcitation(2.85)─├FermionicDoubleExcitation(0.03) ···
 5: ──X─├FermionicDoubleExcitation(2.85)─├FermionicDoubleExcitation(0.03) ···
 6: ────│────────────────────────────────│─────────────────────────────── ···
 7: ────│────────────────────────────────│─────────────────────────────── ···
 8: ─

/Users/s.poyyapakkam/miniconda3/envs/na/lib/python3.11/site-packages/pennylane_lightning/lightning_base/_serialize.py:359: ComplexWarning: Casting complex values to real discards the imaginary part
  coeffs = np.array(coeffs).astype(self.rtype)


The highest operator value is 0.006153108541835377 for operator a⁺(0) a⁺(1) a(7) a(8)
Highest gradient excitation is [0, 1, 7, 8]


In [3]:
energies

[-1655.1697678320475,
 -1655.193248120223,
 -1655.2057005390784,
 -1655.2432085677262,
 -1655.246586348109,
 -1655.2473051516004,
 -1655.2770120048424,
 -1655.2772736485945,
 -1655.2775881515895,
 -1655.3085193509735,
 -1655.3085205452417,
 -1655.3085205452312,
 -1655.3085205452207,
 -1655.3085206323292,
 -1655.3085207398753,
 -1655.308520739864,
 -1655.3085207703803,
 -1655.3085207777965,
 -1655.3085207955514,
 -1655.308520795598,
 -1655.308520798619,
 -1655.3085207986094,
 -1655.3085207985991,
 -1655.308520798589,
 -1655.3085207985787,
 -1655.3085207985675,
 -1655.3085207986085,
 -1655.3085207991637,
 -1655.308520884106,
 -1655.3085208841026,
 -1655.3085208840926,
 -1655.308520884083,
 -1655.3085208840728,
 -1655.3085208840625,
 -1655.3085208840525,
 -1655.308520884043,
 -1655.3085208840328,
 -1655.3085208840223,
 -1655.3085208840118,
 -1655.3085208840014,
 -1655.3085208839912,
 -1655.3085208839805,
 -1655.3085208839702,
 -1655.3085208839589,
 -1655.3085208839493,
 -1655.308520883939